In [1]:
%load_ext autoreload
%autoreload 2
from src import run_exp, statistics, get_graph, split_pos_neg, label_propagation
import networkx as nx
import pandas as pd
from pytorch_lightning import seed_everything
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

In [2]:
# baseline_methods = ['metis', 'louvain', 'eva', 'maxflow_cpp_unweighted', 'maxflow_cpp_weighted', 'gnn', 'pads_cpp']
baseline_methods = ['maxflow_cpp_weighted', 'neg_dsd_cpp', 'gnn', 'pads_cpp']#, 'pads_python_3k_2k', 'pads_python_4k_3k']
datasets = ['Abortion', 'Brexit', 'Election', 'Gun', 'Partisanship', 'Referendum_']
seed_everything(20010302)

Seed set to 20010302


20010302

In [15]:
theta = 0.5
for d in datasets[1:2]:
    print(f"===Dataset {d}===")
    G = get_graph(d)
    # split_pos_neg(G, f'input/datasets/static/{d}', 500)
    timer = {}
    for method in ['maxflow_cpp_weighted', 'neg_dsd_cpp', 'pads_python', 'pads_cpp']:
        if method.startswith('pads_python_'):
            parts = method.split('_')
            if len(parts) == 4:
                size1 = int(parts[2].replace('k', '000'))
                size2 = int(parts[3].replace('k', '000'))
                timer[method], rst = run_exp(G, 'pads_python', dataset=d, theta=theta,
                                           size_ub=(size1, size2), attr_name=method)
        else:
            timer[method], rst = run_exp(G, method, dataset=d, theta=theta)
        print(f"{method} Finished! Time taken: {timer[method]}")

    # G = label_propagation(G, attr_name='pads_cpp', new_attr='pads_cpp_lp', threshold=0.7)

    if not os.path.exists(f'output/results-theta={theta}/{d}/'):
        os.makedirs(f'output/results-theta={theta}/{d}/')
    nx.write_gml(G, f'output/results-theta={theta}/{d}/graph.gml')
    pd.DataFrame(timer.items(), columns=['method', 'time']).to_csv(f'output/results-theta={theta}/{d}/time.csv')

    df_pos, df_neg = statistics(G, f'output/results-theta={theta}/{d}')
    print(df_pos)
    print(df_neg)

===Dataset Brexit===
maxflow_cpp_weighted Finished! Time taken: 0.493
neg_dsd_cpp Finished! Time taken: 0.019993
pads_python Finished! Time taken: 1.355
pads_cpp Finished! Time taken: 0.009636
[('cascade', 3), ('maxflow_cpp_wdsp', 1), ('neg_dsd', 1), ('pads_python', 1), ('pads_cpp', 1)]
[('cascade', 6), ('maxflow_cpp_wdsp', -1), ('neg_dsd', -1), ('pads_python', -1), ('pads_cpp', -1)]
             method  num_nodes  num_edges purity(var) conductance  \
0           cascade       4423    63853.5     0.40819     0.07499   
1  maxflow_cpp_wdsp        389    17207.0     0.03291     0.50850   
2           neg_dsd        388    17350.0     0.03583     0.50857   
3       pads_python        398    17758.5     0.03559     0.50213   
4          pads_cpp        399    17803.5     0.03550     0.50131   

  avg_node_polarity unweighted_density weighted_density  EI-Index  
0           0.47996           14.43669          7.84296  -0.85002  
1           0.64549           44.23393         27.76936  -0.98